In [ ]:

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%%capture

#En la consola ejecutaremos los siguientes comandos.

!curl -fsSL https://ollama.com/install.sh | sh

!nohup ollama serve >output.log 2>&1 &


#!ollama serve &
!ollama pull llama3
!ollama pull all-minilm  #modelo para  el embedding

!pip install langchain
!pip install langchain-ollama
!pip install langchain_community
!pip install langchain_chroma
!pip install langchainhub
!pip install langsmith

import os
from langchain.document_loaders import TextLoader

os.makedirs('/content/drive/MyDrive/NLP/normas_texto', exist_ok=True)

!git clone https://github.com/bettachini/nlpTP.git
!cp -r nlpTP/arn/normas_texto/* /content/drive/MyDrive/NLP/normas_texto/
!ls /content/drive/MyDrive/NLP/normas_texto
# Ruta de la carpeta en tu Google Drive que contiene los archivos .txt
folder_path = '/content/drive/MyDrive/NLP/normas_texto'

# documentos cargados
docs = []
for filename in os.listdir(folder_path):
    if filename.endswith(".txt"):
        file_path = os.path.join(folder_path, filename)
        loader = TextLoader(file_path)  # TextLoader carga archivos de texto
        docs.extend(loader.load())  # Cargar y agregar a la lista de documentos

from langchain import hub
from langchain.vectorstores import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.prompts.chat import ChatPromptTemplate
from langchain_ollama import ChatOllama, OllamaEmbeddings

llm = ChatOllama(
                model = "llama3"
                )
emb = OllamaEmbeddings(model="all-minilm")

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=emb)

retriever = vectorstore.as_retriever()

In [ ]:
template = """
            Sos un experto en seguridad nuclear  dispuesto a responder preguntas sobre las normas y guias emitidas por la Autoridad Regulatoria Nuclear de Argentina.Tu tarea es responder la pregunta provista abajo.

            Tambien se proveen una serie de documentos que un retrieval engine considero mas similares a la pregunta.

            Provees respuestas concisas siguiendo la lógica "pensemos paso a paso" y la siguente estructura, donde DOCUMENTOS es una lista de documentos reglamentarios a partir de los cuales se elabora la respuesta:

            Pregunta:
            {query}

            Respuesta:

            DOCUMENTOS:
            {docs}

            """
#hay que agregar metadata
prompt = ChatPromptTemplate.from_template(template)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {
        "docs": retriever | format_docs,
        "query": RunnablePassthrough()
    }
    | prompt
    | llm
    | StrOutputParser()
)

!pip install rouge-score bert-score
from bert_score import score

from rouge_score import rouge_scorer

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.9 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=874b41d1f628dcbac938862406ee1bc9557f8def51223d3c26519edaacea4b8b
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


# Measuring answer similarity

medidas posibles

BLEU (Bilingual Evaluation Understudy)
ROUGE (Recall-Oriented Understudy for Gisting Evaluation)
METEOR
BERTScore

Bloque para obtener preguntas, pasarlas al modelo, comparar las métricas y guardar en la hoja de cálculo

https://docs.google.com/spreadsheets/d/1ZxNovjFeHOc6vBhhdg6K49TpQd00Y9IiSu_A1pawrys/edit?usp=sharing

In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

# Abrir por titulo
sh = gc.open("GoldStandardGPT4")

# Seleccionar primera hoja
worksheet = sh.sheet1



In [ ]:
worksheet.acell("B1").value

'Pregunta'

In [ ]:
n=17
data_available=[]
for n in range(1,n+1):
  document=worksheet.acell(f"A{n+2}")
  question = worksheet.acell(f'B{n+2}').value
  gs_answer = val = worksheet.acell(f'C{n+2}').value
  data_available.append({"question":question,"gs_answer":gs_answer})


In [ ]:
data_available

[{'question': '¿Cuáles son los estudios previos que deben realizarse para determinar la viabilidad de una nueva planta nuclear en un sitio específico?',
  'gs_answer': 'Pregunta: ¿Cuáles son los estudios previos que deben realizarse para determinar la viabilidad de una nueva planta nuclear en un sitio específico? Respuesta: Antes de la construcción, se deben realizar estudios geológicos, hidrológicos, meteorológicos y de impacto ambiental para evaluar la seguridad y la sostenibilidad del sitio. También es necesario analizar el impacto radiológico en el área circundante.\nDocumentos relacionados: “Norma de Evaluación de Impacto Ambiental para Instalaciones Nucleares” (ARN 3.2).'},
 {'question': '¿Qué requisitos de calificación deben cumplir los operadores de una planta nuclear?',
  'gs_answer': 'Pregunta: ¿Qué requisitos de calificación deben cumplir los operadores de una planta nuclear? Respuesta: Los operadores deben tener una formación específica en ingeniería nuclear, física o un ca

In [ ]:
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

def generate_using_new_template(exp_number):
  important_info=[]
  important_info.append(["Pregunta","GoldStandard","Respuesta","BertSCORE_P","BertSCORE_R","BertSCORE_F1","ROUGE1","ROUGE2","ROUGEL"])
  for n,q in enumerate(data_available):
      #Bertscore
      current_answer=rag_chain.invoke(q["question"])
      gs_answer=q["gs_answer"]
      print(f"Pregunta {n}")
      P, R, F1 = score([current_answer], [gs_answer], lang="es")
      #rouge
      scores = scorer.score(current_answer, gs_answer)

      important_info.append([q["question"],q["gs_answer"],current_answer,str(P.item()),str(R.item()),str(F1.item()),str(scores['rouge1']),str(scores['rouge2']),str(scores['rougeL'])])

  with open(f'/content/drive/MyDrive/NLP/results/results_experiment_{exp_number}.txt', 'w') as f:
    for line in important_info:
        linestr=[str(i) for i in line]
        linea=";".join(linestr)
        f.write(f"{linea}\n")



#Acá modificamos:
Cambiamos el template a gusto y piacere

In [ ]:
template = """
            Sos un experto en seguridad nuclear  dispuesto a responder preguntas sobre las normas y guias emitidas por la Autoridad Regulatoria Nuclear de Argentina.Tu tarea es responder la pregunta provista abajo.

            Tambien se proveen una serie de documentos que un retrieval engine considero mas similares a la pregunta.

            Provees respuestas concisas siguiendo la lógica "pensemos paso a paso" y la siguente estructura, donde DOCUMENTOS es una lista de documentos reglamentarios a partir de los cuales se elabora la respuesta:

            Pregunta:
            {query}

            Respuesta:

            DOCUMENTOS:
            {docs}

            """
#hay que agregar metadata
prompt = ChatPromptTemplate.from_template(template)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {
        "docs": retriever | format_docs,
        "query": RunnablePassthrough()
    }
    | prompt
    | llm
    | StrOutputParser()
)

## Llamado a la función que genera el experimento

In [ ]:
generate_using_new_template(exp_number=2)

Pregunta 0
Pregunta 1
Pregunta 2
Pregunta 3
Pregunta 4
Pregunta 5
Pregunta 6
Pregunta 7
Pregunta 8
Pregunta 9
Pregunta 10
Pregunta 11
Pregunta 12
Pregunta 13
Pregunta 14
Pregunta 15
Pregunta 16


Vamos a probar algo que de peor?

In [ ]:
template = """
            Sos un fanático del rock nacional argentino.Tu tarea es responder la pregunta provista abajo utilizando los
            documentos que un retrieval engine considero mas similares a la pregunta.

            Provees respuestas concisas siguiendo la lógica "pensemos paso a paso" y la siguente estructura, donde DOCUMENTOS es una lista de documentos reglamentarios a partir de los cuales se elabora la respuesta:

            Pregunta:
            {query}

            Respuesta:

            DOCUMENTOS:
            {docs}

            """
#hay que agregar metadata
prompt = ChatPromptTemplate.from_template(template)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {
        "docs": retriever | format_docs,
        "query": RunnablePassthrough()
    }
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
generate_using_new_template(exp_number=3)

Pregunta 0
Pregunta 1
Pregunta 2
Pregunta 3
Pregunta 4
Pregunta 5
Pregunta 6
Pregunta 7
Pregunta 8
Pregunta 9
Pregunta 10
Pregunta 11
Pregunta 12
Pregunta 13
Pregunta 14
Pregunta 15
Pregunta 16
